## Imports

In [288]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, precision_score
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.model_selection import train_test_split
import itertools

## Utils

In [289]:
def generate_hyperparameter_combinations(param_ranges):
    """
    Genera tutte le combinazioni di iperparametri basate su range e step specificati.

    :param param_ranges: Dizionario con i nomi degli iperparametri come chiavi.
                         Ogni valore è una tupla (start, stop, step).
    :return: Lista di dizionari con tutte le combinazioni possibili.
    """
    param_values = {
        key: np.arange(start, stop + step, step)
        for key, (start, stop, step) in param_ranges.items()
    }
    
    param_combinations = list(itertools.product(*param_values.values()))
    return [
        dict(zip(param_values.keys(), combination))
        for combination in param_combinations
    ]

## Data load

In [290]:
# One-hot encoding
encoder = OneHotEncoder(categories='auto', sparse_output=False)

# Carica i file di addestramento e test per ciascun dataset dal percorso specificato
monk1_train = pd.read_csv('../Datasets/Monks/monks-1.train', sep='\s+', header=None)
monk1_test = pd.read_csv('../Datasets/Monks/monks-1.test', sep='\s+', header=None)

monk2_train = pd.read_csv('../Datasets/Monks/monks-2.train', sep='\s+', header=None)
monk2_test = pd.read_csv('../Datasets/Monks/monks-2.test', sep='\s+', header=None)

monk3_train = pd.read_csv('../Datasets/Monks/monks-3.train', sep='\s+', header=None)
monk3_test = pd.read_csv('../Datasets/Monks/monks-3.test', sep='\s+', header=None)


# Lista per memorizzare i dataset trasformati
monks_train = []
monks_test = []


# Dataset monk1
X1_train = monk1_train.iloc[:, 1:7].values  # Caratteristiche
y1_train = monk1_train.iloc[:, 0].values    # Etichette

X1_test = monk1_test.iloc[:, 1:7].values
y1_test = monk1_test.iloc[:, 0].values

# Applicazione dell'encoder a monk1
X1_train_encoded = encoder.fit_transform(X1_train)  # Fit e trasformazione sui dati di training
X1_test_encoded = encoder.transform(X1_test)        # Solo trasformazione sui dati di test

monks_train.append((X1_train_encoded, y1_train))
monks_test.append((X1_test_encoded, y1_test))

# Dataset monk2
X2_train = monk2_train.iloc[:, 1:7].values
y2_train = monk2_train.iloc[:, 0].values

X2_test = monk2_test.iloc[:, 1:7].values
y2_test = monk2_test.iloc[:, 0].values

# Applicazione dell'encoder a monk2
X2_train_encoded = encoder.fit_transform(X2_train)
X2_test_encoded = encoder.transform(X2_test)

monks_train.append((X2_train_encoded, y2_train))
monks_test.append((X2_test_encoded, y2_test))

# Dataset monk3
X3_train = monk3_train.iloc[:, 1:7].values
y3_train = monk3_train.iloc[:, 0].values

X3_test = monk3_test.iloc[:, 1:7].values
y3_test = monk3_test.iloc[:, 0].values

# Applicazione dell'encoder a monk3
X3_train_encoded = encoder.fit_transform(X3_train)
X3_test_encoded = encoder.transform(X3_test)

monks_train.append((X3_train_encoded, y3_train))
monks_test.append((X3_test_encoded, y3_test))

## Model creation

In [291]:
def create_SVM(C = 100, type = 'rbf'):
    return SVC(kernel= type, C=C, random_state=42)

## K-fold cross validation

In [292]:
def k_fold_cross_validation(data, labels, type = 'rbf',params=None):
    # 3. Configurazione della k-fold cross-validation
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # 4. Ciclo di cross-validation
    fold_no = 1
    accuracy_per_fold = []
    for train_index, val_index in kfold.split(data, labels):
        
        # Suddivisione del dataset
        X_train, X_val = data[train_index], data[val_index]
        y_train, y_val = labels[train_index], labels[val_index]

        # Creazione della rete neurale
        model = create_SVM(C=params['C'],type = type)

        # Addestramento con EarlyStopping
        model.fit(X_train, y_train)

        pred = model.predict(X_val)
        # Prendi il miglior score (l'accuratezza di validazione massima)
        score =accuracy_score(pred, y_val)
        accuracy_per_fold.append(score)    
        fold_no += 1

    avg_score = np.mean(accuracy_per_fold)


    _ , X_val, _ , y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

     # Creazione della rete neurale
    model = create_SVM(C=params['C'], type = type)

    # Addestramento con EarlyStopping
    model.fit(data, labels)

    return avg_score, model

## Greed search

In [293]:
def greed_search(data, labels, type = 'rbf', param_grid = None):
    best_scores = []  # Usa una lista normale per memorizzare i punteggi
    best_params_list = []  # Lista per le configurazioni
    best_models = []  # Lista per i modelli
    
    for params in param_grid:
        
        score, model = k_fold_cross_validation(data, labels, type, params=params)

        # Aggiungi i risultati alla lista
        best_scores.append(score)
        best_params_list.append(params)
        best_models.append(model)

        # Ordina la lista dei punteggi e mantieni solo i migliori 10
        sorted_indices = np.argsort(best_scores)[::-1]  # Ordina i punteggi in ordine decrescente
        best_scores = [best_scores[i] for i in sorted_indices][:5]  # Usa la lista e mantieni i top 10
        best_params_list = [best_params_list[i] for i in sorted_indices][:5]
        best_models = [best_models[i] for i in sorted_indices][:5]

    return best_scores, best_params_list, best_models

## Model selection

In [294]:
def selection(data, labels):

    # Definizione dei range degli iperparametri
    param_ranges = {
        "C": (1, 1000, 10),  # Da 0.01 a 0.5 con step di 0.05
    }

    print("Generazione delle combinazioni di iperparametri...")
    param_grid = []
    param_grid = generate_hyperparameter_combinations(param_ranges)


    best_scores = []  # Usa una lista normale per memorizzare i punteggi
    best_params_list = []  # Lista per le configurazioni
    best_models = []  # Lista per i modelli
    best_types = []


    #np.random.choice(param_grid, size=12, replace=False)
    # Seleziona 100 elementi a caso senza rimpiazzamento
    types = ['linear', 'poly', 'rbf', 'sigmoid']
    for type in types:
        actual_scores , actual_params_list, actual_models = greed_search(data, labels, type, param_grid)

        best_scores.extend(actual_scores)
        best_params_list.extend(actual_params_list)
        best_models.extend(actual_models)
        best_types.extend([type]*len(actual_scores))

         # Ordina la lista dei punteggi e mantieni solo i migliori 10
        sorted_indices = np.argsort(best_scores)[::-1]  # Ordina i punteggi in ordine decrescente
        best_scores = [best_scores[i] for i in sorted_indices][:5]  # Usa la lista e mantieni i top 10
        best_params_list = [best_params_list[i] for i in sorted_indices][:5]
        best_models = [best_models[i] for i in sorted_indices][:5]
        best_types = [best_types[i] for i in sorted_indices][:5]
    
    for i in range(len(best_scores)):
        print(f"Score: {best_scores[i]}, type: {best_types[i]}, parametri: {best_params_list[i]}")

    return best_scores, best_params_list, best_models


In [295]:
print("-----------------MONK1-----------------")
_, _,best_models_monk_1 = selection(monks_train[0][0], monks_train[0][1])
print("-----------------MONK2-----------------")
_, _,best_models_monk_2 = selection(monks_train[1][0], monks_train[1][1])
print("-----------------MONK3-----------------")
_, _,best_models_monk_3 = selection(monks_train[2][0], monks_train[2][1])

-----------------MONK1-----------------
Generazione delle combinazioni di iperparametri...
Score: 0.9596666666666666, type: poly, parametri: {'C': 31}
Score: 0.9596666666666666, type: poly, parametri: {'C': 11}
Score: 0.9596666666666666, type: poly, parametri: {'C': 21}
Score: 0.9596666666666666, type: poly, parametri: {'C': 41}
Score: 0.9596666666666666, type: poly, parametri: {'C': 1001}
-----------------MONK2-----------------
Generazione delle combinazioni di iperparametri...
Score: 0.7103386809269162, type: poly, parametri: {'C': 31}
Score: 0.7103386809269162, type: poly, parametri: {'C': 11}
Score: 0.7103386809269162, type: poly, parametri: {'C': 21}
Score: 0.7103386809269162, type: poly, parametri: {'C': 41}
Score: 0.7103386809269162, type: poly, parametri: {'C': 1001}
-----------------MONK3-----------------
Generazione delle combinazioni di iperparametri...
Score: 0.9343333333333333, type: linear, parametri: {'C': 21}
Score: 0.9343333333333333, type: linear, parametri: {'C': 1}


## Model assessment

In [296]:
# 5. Valutazione del modello
y1_pred = best_models_monk_1[0].predict(X1_test_encoded)

# 6. Report dei risultati
print("Accuracy:", accuracy_score(y1_test, y1_pred))
print("\nClassification Report:\n", classification_report(y1_test, y1_pred))

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       216
           1       1.00      1.00      1.00       216

    accuracy                           1.00       432
   macro avg       1.00      1.00      1.00       432
weighted avg       1.00      1.00      1.00       432



In [297]:
# 5. Valutazione del modello
y2_pred = best_models_monk_2[0].predict(X2_test_encoded)

# 6. Report dei risultati
print("Accuracy:", accuracy_score(y2_test, y2_pred))
print("\nClassification Report:\n", classification_report(y2_test, y2_pred))

Accuracy: 0.7731481481481481

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.77      0.82       290
           1       0.62      0.77      0.69       142

    accuracy                           0.77       432
   macro avg       0.75      0.77      0.76       432
weighted avg       0.79      0.77      0.78       432



In [298]:
y3_pred = best_models_monk_3[0].predict(X3_test_encoded)

# 6. Report dei risultati
print("Accuracy:", accuracy_score(y3_test, y3_pred))
print("\nClassification Report:\n", classification_report(y3_test, y3_pred))

Accuracy: 0.9722222222222222

Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       204
           1       1.00      0.95      0.97       228

    accuracy                           0.97       432
   macro avg       0.97      0.97      0.97       432
weighted avg       0.97      0.97      0.97       432

